# Getting the embeddings

> This notebook gets the embeddings (or latent space) from a multivariate time series 
given by a encoder (e.g., autoencoder)

In [1]:
from dvats.all import *
from tsai.data.preparation import SlidingWindow
from fastcore.all import *
import wandb
wandb_api = wandb.Api()
from yaml import load, FullLoader

## Config parameters

Put here everything that could be needed if this notebook

In [2]:
import utils.config as cfg
config, job_type = cfg.get_artifact_config_embeddings(True)

Current: /home/macu/work/nbs_pipeline
yml: ./config/03-embeddings.yaml
yml: ./config/03-embeddings.yaml
Getting content./config/03-embeddings.yaml
... About to replace includes with content
Load content./config/03-embeddings.yaml
enc_artifact: mi-santamaria/deepvats/mvp:latest


## Run

In [3]:
import os
path = os.path.expanduser("~/work/nbs_pipeline/")
name="03_embeddings"
os.environ["WANDB_NOTEBOOK_NAME"] = path+name+".ipynb"
runname=name
print("runname: "+runname)

runname: 03_embeddings


In [4]:
run = wandb.init(
    entity      = config.wandb_entity,
    project     = config.wandb_project if config.use_wandb else 'work-nbs', 
    group       = config.wandb_group,
    job_type    = job_type,
    mode        = 'online' if config.use_wandb else 'disabled',
    anonymous   = 'never' if config.use_wandb else 'must',
    config      = config,
    resume      = 'allow',
    name        = runname
)

wandb: Currently logged in as: mi-santamaria. Use `wandb login --relogin` to force relogin


'stream.Stream' object attribute 'write' is read-only


In [5]:
# Botch to use artifacts offline
artifacts_gettr = run.use_artifact if config.use_wandb else wandb_api.artifact

Restore the encoder model and its associated configuration

In [6]:
enc_artifact = artifacts_gettr(config.enc_artifact, type='learner')

In [7]:
# TODO: This only works when you run it two timeS! WTF?
try:
    enc_learner = enc_artifact.to_obj()
except:
    enc_learner = enc_artifact.to_obj()

wandb:   1 of 1 files downloaded.  


Restore the dataset artifact used for training the encoder. Even if we do not compute the dimensionality reduction over this dataset, we need to know the metadata of the encoder training set, to check that 
it matches with the dataset that we want to reduce.

In [8]:
enc_run = enc_artifact.logged_by()
enc_artifact_train = artifacts_gettr(enc_run.config['train_artifact'], type='dataset')
enc_artifact_train.name

'Occupancy_estimation:v3'

Now we specify the dataset artifact that we want to get the embeddings from. If no 
artifact is defined, the artifact to reduce will be the one used for validate the encoder.

In [9]:
input_ar_name = ifnone(config.input_ar, 
                       f'{enc_artifact_train.entity}/{enc_artifact_train.project}/{enc_artifact_train.name}')
wandb.config.update({'input_ar': input_ar_name}, allow_val_change=True)
input_ar = artifacts_gettr(input_ar_name)
input_ar.name

'Occupancy_estimation:v3'

In [10]:
df = input_ar.to_df()
df

wandb:   1 of 1 files downloaded.  


,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
Timestamp,,,,,,,,,,,,,,,,,
2017-12-22 10:49:41,24.94,24.75,24.56,25.38,121,34,53,40,0.08,0.19,0.06,0.06,390,0.769231,0,0,1
2017-12-22 10:50:12,24.94,24.75,24.56,25.44,121,33,53,40,0.93,0.05,0.06,0.06,390,0.646154,0,0,1
2017-12-22 10:50:42,25.00,24.75,24.50,25.44,121,34,53,40,0.43,0.11,0.08,0.06,390,0.519231,0,0,1
2017-12-22 10:51:13,25.00,24.75,24.56,25.44,121,34,53,40,0.41,0.10,0.10,0.09,390,0.388462,0,0,1
2017-12-22 10:51:44,25.00,24.75,24.56,25.44,121,34,54,40,0.18,0.06,0.06,0.06,390,0.253846,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-01-11 08:58:07,25.06,25.13,24.69,25.31,6,7,33,22,0.09,0.04,0.06,0.08,345,0.000000,0,0,0
2018-01-11 08:58:37,25.06,25.06,24.69,25.25,6,7,34,22,0.07,0.05,0.05,0.08,345,0.000000,0,0,0
2018-01-11 08:59:08,25.13,25.06,24.69,25.25,6,7,34,22,0.11,0.05,0.06,0.08,345,0.000000,0,0,0


In [11]:
df.shape

(8828, 17)

In [12]:
enc_input, _ = SlidingWindow(window_len=enc_run.config['w'], 
                             stride=enc_run.config['stride'], 
                             get_y=[])(df)
enc_input.shape

(8799, 17, 30)

In [13]:
embs = get_enc_embs(enc_input, enc_learner, cpu=config.cpu, to_numpy=True)

--> Use CUDA
Use CUDA -->
--> Nested atrr
--> Embs


<class 'RuntimeError'>: Given groups=1, weight of size [32, 4, 1], expected input[1, 17, 30] to have 4 channels, but got 17 channels instead

In [ ]:
if config.use_wandb: 
    run.log_artifact(ReferenceArtifact(embs, 'embeddings', metadata=dict(run.config)), 
                     aliases=f'run-{run.project}-{run.id}')

In [ ]:
run.finish()